https://dacon.io/competitions/official/236035/codeshare/7413?page=2&dtype=recent

### 유전체 정보 품종 분류 AI 경진대회

- 개체 정보와 사전에 구성된 15개의 SNP 정보를 바탕으로 품종 분류 모델을 개발

  id : 개체 고유 ID

- 개체정보

  father : 개체의 가계 고유 번호 (0 : Unknown)

  mother : 개체의 모계 고유 번호 (0 : Unknown)

  gender : 개체 성별 (0 : Unknown, 1 : female, 2 : male)

  trait : 개체 표현형 정보 

  15개의 SNP 정보 : SNP_01 ~ SNP_15

  class : 개체의 품종 (A,B,C)

- 15개의 SNP 세부 정보

  name : SNP 명

  chrom : 염색체 정보

  cm : Genetic distance
  
  pos : 각 마커의 유전체상 위치 정보

In [4]:
import pandas as pd

info = pd.read_csv('/content/snp_info.csv')
info

,SNP_id,name,chrom,cm,pos
0,SNP_01,BTA-19852-no-rs,2,67.05460,42986890
1,SNP_02,ARS-USMARC-Parent-DQ647190-rs29013632,6,31.15670,13897068
2,SNP_03,ARS-BFGL-NGS-117009,6,68.28920,44649549
3,SNP_04,ARS-BFGL-NGS-60567,6,77.87490,53826064
4,SNP_05,BovineHD0600017032,6,80.50150,61779512
5,SNP_06,BovineHD0600017424,6,80.59540,63048481
6,SNP_07,Hapmap49442-BTA-111073,6,80.78000,64037334
7,SNP_08,BovineHD0600018638,6,82.68560,67510588
8,SNP_09,ARS-BFGL-NGS-37727,6,86.87400,73092782
9,SNP_10,BTB-01558306,7,62.06920,40827112


https://dacon.io/competitions/official/236035/codeshare?page=2&dtype=recent&ptype=pub&keyword=

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random
import os
import numpy as np

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [6]:
train.head(3)

,id,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,...,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
0,TRAIN_000,0,0,0,2,G G,A G,A A,G A,C A,...,A A,G G,A A,G G,A G,A A,A A,A A,A A,B
1,TRAIN_001,0,0,0,2,A G,A G,C A,A A,A A,...,A A,G A,A A,A G,A A,G A,G G,A A,A A,C
2,TRAIN_002,0,0,0,2,G G,G G,A A,G A,C C,...,A A,G A,G A,A G,A A,A A,A A,A A,A A,B


In [7]:
test.head(3)

,id,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
0,TEST_000,0,0,0,1,A G,G G,A A,G A,A A,A G,G G,G A,G A,A G,A G,G A,G G,C A,G A
1,TEST_001,0,0,0,2,G G,A G,C C,G G,C C,A A,A A,A A,A A,G G,A G,A A,A A,A A,A A
2,TEST_002,0,0,0,2,G G,A G,A A,A A,C A,A G,A A,A A,A A,A G,A A,G A,G G,A A,G G


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262 entries, 0 to 261
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      262 non-null    object
 1   father  262 non-null    int64 
 2   mother  262 non-null    int64 
 3   gender  262 non-null    int64 
 4   trait   262 non-null    int64 
 5   SNP_01  262 non-null    object
 6   SNP_02  262 non-null    object
 7   SNP_03  262 non-null    object
 8   SNP_04  262 non-null    object
 9   SNP_05  262 non-null    object
 10  SNP_06  262 non-null    object
 11  SNP_07  262 non-null    object
 12  SNP_08  262 non-null    object
 13  SNP_09  262 non-null    object
 14  SNP_10  262 non-null    object
 15  SNP_11  262 non-null    object
 16  SNP_12  262 non-null    object
 17  SNP_13  262 non-null    object
 18  SNP_14  262 non-null    object
 19  SNP_15  262 non-null    object
 20  class   262 non-null    object
dtypes: int64(4), object(17)
memory usage: 43.1+ KB


In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      175 non-null    object
 1   father  175 non-null    int64 
 2   mother  175 non-null    int64 
 3   gender  175 non-null    int64 
 4   trait   175 non-null    int64 
 5   SNP_01  175 non-null    object
 6   SNP_02  175 non-null    object
 7   SNP_03  175 non-null    object
 8   SNP_04  175 non-null    object
 9   SNP_05  175 non-null    object
 10  SNP_06  175 non-null    object
 11  SNP_07  175 non-null    object
 12  SNP_08  175 non-null    object
 13  SNP_09  175 non-null    object
 14  SNP_10  175 non-null    object
 15  SNP_11  175 non-null    object
 16  SNP_12  175 non-null    object
 17  SNP_13  175 non-null    object
 18  SNP_14  175 non-null    object
 19  SNP_15  175 non-null    object
dtypes: int64(4), object(16)
memory usage: 27.5+ KB


In [10]:
train.isna().sum()

id        0
father    0
mother    0
gender    0
trait     0
SNP_01    0
SNP_02    0
SNP_03    0
SNP_04    0
SNP_05    0
SNP_06    0
SNP_07    0
SNP_08    0
SNP_09    0
SNP_10    0
SNP_11    0
SNP_12    0
SNP_13    0
SNP_14    0
SNP_15    0
class     0
dtype: int64

In [11]:
test.isna().sum()

id        0
father    0
mother    0
gender    0
trait     0
SNP_01    0
SNP_02    0
SNP_03    0
SNP_04    0
SNP_05    0
SNP_06    0
SNP_07    0
SNP_08    0
SNP_09    0
SNP_10    0
SNP_11    0
SNP_12    0
SNP_13    0
SNP_14    0
SNP_15    0
dtype: int64

In [12]:
train.nunique()

id        262
father      1
mother      1
gender      1
trait       2
SNP_01      3
SNP_02      3
SNP_03      3
SNP_04      3
SNP_05      3
SNP_06      3
SNP_07      3
SNP_08      3
SNP_09      3
SNP_10      3
SNP_11      3
SNP_12      3
SNP_13      3
SNP_14      3
SNP_15      3
class       3
dtype: int64

https://mz-moonzoo.tistory.com/5

#### AutoML이란

- 머신러닝 모델을 학습하고, 배포하는 과정을 자동화하는 기술 혹은 도구

- 머신러닝 전문가가 수행하던 영역 중 정형화된 과정을 자동화하여 업무 효율을 높여줌


#### AutoML의 종류 및 특징

- OSS (Open Source Software)

  - 오픈소스 기반의 라이브러리 형태로 제공되는 AutoML 기능

    ex) Sklearn의 Auto-sklearn, TPOT(구조화된 데이터에 적합)

    Keras 기반의 AutoKeras, Tensorflow 기반의 AdaNet(텍스트, 이미지 처리같은 딥러닝 모델에 적합, 신경망 구조 탐색)


- Cloud Provider Solution

  - 주요 클라우드 서비스에서 제공하는 클라우드 환경 AutoML

  - 클라우드 환경이라 필요한 만큼 자원 할당이 가능, 데이터 전처리부터 모델링, 후처리 기능까지 제공해서 전체 프로세스를 구현할 수 있음

    ex) Google의 Cloud AutoML, Amazon의 Sagemaker Autopilot, Microsoft의 Azure

- Enterprise solution

  - AutoML 서비스 제공을 목적으로 만들어진 전문 플랫폼

    ex) DataRobot과 H20

https://data-minggeul.tistory.com/12


#### 1. PyCaret

- 머신러닝 워크플로우를 자동화하는 Python의 AutoML 라이브러리

- Classification, Regression, Clustering등의 Task에서 사용하는 여러모델들을 동일한 환경에서 한번에 한줄의 코드로 실행할 수 있도록 자동화함

- 기존 Scikit-learn, XGBoost, LightGBM, spaCy 등 다양한 머신러닝 라이브러리를 ML High-Level API로 제작

- 데이터 분석 및 머신러닝 모델 성능 비교, 각 모델 별로 튜닝 가능, Log 생성 및 이력 확인 가능

https://pycaret.gitbook.io/docs/

In [13]:
!pip install pycaret
!pip install markupsafe==2.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.6/483.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.4 MB/s eta 0:00:00

데이터 준비

- pycaret.classification.setup(): pycaret을 사용하기 위해 setup으로 데이터 로드

모델 생성 및 비교

- pycaret.classification.compare_models(): 머신러닝 모델 적용 및 비교

- pycaret.classification.create_model(): 머신러닝 모델 선택 및 생성

모델 최적화

- pycaret.classification.tune_model(): 하이퍼파라미터 최적화

- pycaret.classification.ensemble_model(): ensemble 기법

- pycaret.classification.blend_models(): Voting 기법

학습된 모델 분석

- pycaret.classification.predict_model: 테스트 데이터셋으로 모델 평가

- pycaret.classification.evaluate_model(): 모델 성능평가 인터페이스 제공

- pycaret.classification.plot_model(): 모델 성능평가 시각화

- pycaret.classification.interpret_model(): 모델 파라미터 영향도 시각화

https://pycaret.readthedocs.io/en/latest/api/classification.html

In [14]:
train=train.iloc[:, 4:]
test=test.iloc[:, 4:]

In [15]:
train.head(3)

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
0,2,G G,A G,A A,G A,C A,A A,A A,G G,A A,G G,A G,A A,A A,A A,A A,B
1,2,A G,A G,C A,A A,A A,A G,A A,G A,A A,A G,A A,G A,G G,A A,A A,C
2,2,G G,G G,A A,G A,C C,G G,A A,G A,G A,A G,A A,A A,A A,A A,A A,B


##### 1. 데이터 준비

- 자동으로 LabelEncoding 진행

- pycaret setup함수를 사용한다면 복잡했던 데이터 전처리를 한줄의 코드로 간편하게 진행할 수 있다.

< setup() 파라미터 >

- data : Input data를 입력. Train, Test를 별도로 분리하지 않고 입력해주면 뒤의 train_size에 입력한 비율대로 Train,Test를 split 함.

- target : 데이터셋에서 최종적으로 예측해야 하는 Column을 설정해주는 파라미터. data에서 지정해준 input data에 해당 column이 존재해야 가능함.

- session_id : Random seed를 설정해주는 부분. 반복 실행을 진행 했을 때 동일한 결과를 나오도록 함.

- normalize : 데이터에 정규화를 할 것인지 True/False로 선택함.

- normalize_method : normalize를 True인 경우 어떤 방식으로 정규화를 진행할 것인지를 정함.

 ('minmax','maxabs','robust')

- transformation : Power Transformation을 통해서 데이터 샘플들의 분포가 가우시안 분포(정규 분포)에 더 가까워지도록 처리해주는 과정.

- fold_strategy : Fold starategy전략을 선택해서 설정할 수 있는 하이퍼파라미터. pycaret은 기본적으로 10-fold Cross-Validation을 수행함


In [16]:
from pycaret.classification import *

setup_clf = setup(data=train, target='class', train_size=0.8, session_id=777)

,Description,Value
0,Session id,777
1,Target,class
2,Target type,Multiclass
3,Target mapping,"A: 0, B: 1, C: 2"
4,Original data shape,"(262, 17)"
5,Transformed data shape,"(262, 47)"
6,Transformed train set shape,"(209, 47)"
7,Transformed test set shape,"(53, 47)"
8,Numeric features,1
9,Categorical features,15


##### 2.모델 비교

setup함수를 진행하고 models()를 실행해보면 사용가능한 모델을 볼 수 있음

In [17]:
models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


compare_model()로 models()에서 사용가능한 모델의 성능을 비교해볼 수 있음

< 파라미터 >

- n_select : 학습에 수행한 모델을 n_select에서 설정해준 수만큼 성능 순서대로 모델을 저장해줌.

- sort : Metric의 이름을 입력. 어떤 metric의 기준으로 성능을 sort할지 지정해줌.

- include : 어떤 모델들을 비교할지 설정해주는 함수. 여기서 특정 모델을 포함할 수 있음.

- exclude : include와 반대로 특정 모델을 제거하고 비교할 수 있음.

- fold : fold를 지정.

In [22]:
model = compare_models(sort='F1', fold=3, n_select=5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9379,0.9881,0.9379,0.9403,0.9375,0.9043,0.9060,0.6000
lr,Logistic Regression,0.9283,0.9866,0.9283,0.9304,0.9282,0.8898,0.8911,0.4600
lda,Linear Discriminant Analysis,0.9235,0.9870,0.9235,0.9248,0.9231,0.8822,0.8833,0.3700
rf,Random Forest Classifier,0.9235,0.9885,0.9235,0.9283,0.9228,0.8820,0.8853,1.0000
xgboost,Extreme Gradient Boosting,0.9140,0.9810,0.9140,0.9194,0.9135,0.8677,0.8711,0.3600
gbc,Gradient Boosting Classifier,0.9139,0.9838,0.9139,0.9204,0.9134,0.8675,0.8716,0.7267
svm,SVM - Linear Kernel,0.9091,0.0000,0.9091,0.9102,0.9089,0.8601,0.8610,0.3000
ridge,Ridge Classifier,0.9092,0.0000,0.9092,0.9109,0.9086,0.8601,0.8616,0.3100
knn,K Neighbors Classifier,0.9044,0.9782,0.9044,0.9079,0.9027,0.8518,0.8552,0.2767
lightgbm,Light Gradient Boosting Machine,0.8996,0.9836,0.8996,0.9064,0.8984,0.8450,0.8497,0.3567


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

##### 3.모델 튜닝 및 앙상블

- 모델 튜닝

앞에서 저장된 best 5 model을 튜닝해보자

< tuned_model() 파라미터 >

- n_iter : 값을 크게 해줄수록 성능이 더 좋아질 수 있지만 많은 시간이 걸리므로 사용하려는 task에 맞게 설정해줘야 함.

- optimizer : 튜닝을 할 때 어떤 Metric을 기준으로 모델을 선별할 것인지 결정하는 하이퍼파라미터. 

In [23]:
tuned_model = [tune_model(i) for i in model]

#blended_model = blend_models(estimator_list=tuned_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,0.9048,0.9778,0.9048,0.9221,0.9000,0.8511,0.8639
2,0.8095,0.9468,0.8095,0.8095,0.8095,0.7083,0.7083
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,0.9524,1.0000,0.9524,0.9571,0.9515,0.9263,0.9297
5,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,0.9048,0.9552,0.9048,0.9048,0.9048,0.8531,0.8531
7,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,0.9524,0.9746,0.9524,0.9571,0.9515,0.9263,0.9297
2,0.8571,0.9643,0.8571,0.8648,0.8584,0.7835,0.7862
3,0.9524,1.0000,0.9524,0.9571,0.9515,0.9263,0.9297
4,0.9048,0.9825,0.9048,0.9048,0.9048,0.8542,0.8542
5,0.9524,1.0000,0.9524,0.9583,0.9526,0.9271,0.9303
6,0.9048,0.9632,0.9048,0.9048,0.9048,0.8531,0.8531
7,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,0.9524,0.9746,0.9524,0.9571,0.9515,0.9263,0.9297
2,0.8571,0.9429,0.8571,0.8648,0.8584,0.7835,0.7862
3,0.9524,1.0000,0.9524,0.9571,0.9515,0.9263,0.9297
4,0.9524,1.0000,0.9524,0.9571,0.9515,0.9263,0.9297
5,0.9524,1.0000,0.9524,0.9583,0.9526,0.9271,0.9303
6,0.8571,0.9484,0.8571,0.8587,0.8554,0.7782,0.7810
7,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


- 모델 앙상블

  상관관계가 낮은 여러 모델을 ensemble한다면 좀더 일반화된 모델을 만들고 overfitting을 줄일 수 있음.

< blend_models() 파라미터 >

- estimatior_list : 사용하고자 하는 모델 리스트를 넣어준다. 여기서는 앞서 하이퍼파라미터 튜닝을 진행한 5개의 모델을 넣어주자

- fold :  fold 설정

- method : soft와 hard가 있는데 soft가 성능이 좀 더 좋다고 함. task에 따라 다름

- optimize :평가하고자하는 metric을 지정.

In [24]:
blended = blend_models(estimator_list = tuned_model, fold=10,
                       method = 'soft', optimize='F1')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,0.9524,0.9778,0.9524,0.9571,0.9515,0.9263,0.9297
2,0.8571,0.9571,0.8571,0.8648,0.8584,0.7835,0.7862
3,0.9524,1.0000,0.9524,0.9571,0.9515,0.9263,0.9297
4,0.9524,1.0000,0.9524,0.9571,0.9515,0.9263,0.9297
5,0.9524,1.0000,0.9524,0.9583,0.9526,0.9271,0.9303
6,0.9048,0.9371,0.9048,0.9048,0.9048,0.8531,0.8531
7,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [25]:
tuned_model

[ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight={},
                      criterion='gini', max_depth=7, max_features='log2',
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0005, min_samples_leaf=4,
                      min_samples_split=10, min_weight_fraction_leaf=0.0,
                      n_estimators=230, n_jobs=-1, oob_score=False,
                      random_state=777, verbose=0, warm_start=False),
 LogisticRegression(C=1.79, class_weight='balanced', dual=False,
                    fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                    max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=777, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 LinearDiscriminantAnalysis(covariance_estimator=None, n_components=None,
                            priors=None, shrinkage=0.4, solver='eigen',
                         

In [26]:
blended = blend_models(estimator_list = tuned_model,
                       fold = 10,
                       method = 'soft',
                       optimize='F1',
                       )

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,0.9524,0.9778,0.9524,0.9571,0.9515,0.9263,0.9297
2,0.8571,0.9571,0.8571,0.8648,0.8584,0.7835,0.7862
3,0.9524,1.0000,0.9524,0.9571,0.9515,0.9263,0.9297
4,0.9524,1.0000,0.9524,0.9571,0.9515,0.9263,0.9297
5,0.9524,1.0000,0.9524,0.9583,0.9526,0.9271,0.9303
6,0.9048,0.9371,0.9048,0.9048,0.9048,0.8531,0.8531
7,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
8,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

##### 4. 모델 성능 평가

최종적으로 사용할 모델을 finalize_model()안에 넣어주면 된다

evaluate_model() 함수로는 feature_importance 등 다양한 기능을 확인할 수 있음

In [27]:
final_model = finalize_model(blended)
evaluate_model(final_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

<Figure size 800x550 with 0 Axes>

##### 5. 예측하기

predict_model()에 예측하고자 하는 test 데이터를 넣어주면 label과 score가 나옴

In [32]:
prediction = predict_model(final_model, data=test)

In [33]:
prediction

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,prediction_label,prediction_score
0,1,A G,G G,A A,G A,A A,A G,G G,G A,G A,A G,A G,G A,G G,C A,G A,A,0.9197
1,2,G G,A G,C C,G G,C C,A A,A A,A A,A A,G G,A G,A A,A A,A A,A A,B,0.9065
2,2,G G,A G,A A,A A,C A,A G,A A,A A,A A,A G,A A,G A,G G,A A,G G,C,0.9164
3,2,G G,A G,C A,A A,C C,A A,A A,A A,A A,G G,A A,G A,A G,A A,A A,B,0.7039
4,1,A A,G G,A A,G G,A A,G G,G G,A A,G G,A G,G G,G A,G G,A A,G G,A,0.9684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,2,A G,G G,C C,A A,C A,A G,A A,G G,A A,G G,G G,A A,A A,A A,G A,B,0.8649
171,2,G G,A A,A A,A A,C A,A G,A A,A A,A A,A G,A A,A A,A G,A A,G A,C,0.9314
172,2,G G,A A,A A,A A,C A,A G,A A,A A,A A,G G,A G,A A,A G,A A,G G,C,0.8171
173,2,A G,G G,C A,G A,C C,G G,A A,G A,A A,G G,A G,A A,A A,A A,A A,B,0.9460
